# Creation of a wildfire hazard based on MODIS BA
Make sure to have installed both climada_python (v.4.0.1) and climada_petals (branch feature/wildfire) before running this code. You will also need MODIS BA data saved in the directory specified in 'dir_data/0_original_MODIS'. 

In [6]:
import pathlib
import sys
dir_data = '/cluster/work/climate/carmenst/Data/Wildfires/Input/MODIS_tiles'
dir_code = str(pathlib.Path().resolve())

In [ ]:
import climada_petals.hazard.wildfire_tiles as fct_tiles 

In [2]:
# ONLY NECESSARY THE VERY FIRST TIME OR WHEN CHANGING THE DATA STRUCTURE
sys.path.append(dir_code)
# CONFIG FILE
fct_tiles.create_config_MODIS(dir_code, dir_data)

In [8]:
import json
with open('config_MODIS.json', 'r') as file:
    config_MODIS = json.load(file)
sys.path.append(config_MODIS['dir_code'])

variable='Burn Date'

In [10]:
import ast

from climada.hazard import Hazard
import climada_petals.hazard.wildfire as wf_util

In [5]:
txt_tiles = dir_code+'valid_tiles.txt'
with open(txt_tiles, 'r') as file:
    list_tiles = ast.literal_eval(file.read()) 

In [6]:
fct_tiles.create_tile_haz(config_MODIS, list_tiles, variable)

In [7]:
global_haz = fct_tiles.create_global_haz(config_MODIS, variable, (2002, 2025), 20)
filename_save = dir_data+'/'+'global_wf_2002-2025.hdf'
global_haz.write_hdf5(filename_save) 

Chunk:  0
Appended hazard for tile  0 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preprocessing/tiles_hazard/hazard_Burn Date_h01v10.hdf
Appended hazard for tile  1 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preprocessing/tiles_hazard/hazard_Burn Date_h03v06.hdf
Appended hazard for tile  2 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preprocessing/tiles_hazard/hazard_Burn Date_h03v07.hdf
Appended hazard for tile  3 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preprocessing/tiles_hazard/hazard_Burn Date_h05v13.hdf
Appended hazard for tile  4 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preprocessing/tiles_hazard/hazard_Burn Date_h07v05.hdf
Appended hazard for tile  5 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preprocessing/tiles_hazard/hazard_Burn Date_h07v06.hdf
Appended hazard for tile  6 /cluster/work/climate/carmenst/Innosuisse_wildfires/data/1_MODIS_preproces

In [5]:
# REAGGREGATE IN TIME
#global_haz = Hazard.from_hdf5(dir_data+'/'+'global_wf_2002-2025.hdf')
haz_month = wf_util.sum_haz_per_timestep(global_haz, 'season_StateOfWF', filename=None)
haz_month.write_hdf5(dir_data+'/'+'seasonal_global_wf_2002-2025.hdf') 

NameError: name 'global_haz' is not defined

In [11]:
# REAGGREGATE IN SPACE 
from climada.util.api_client import Client
client = Client()
target_centroids =  client.get_centroids(extent=(-180, 180, -90, 90))
threshold = 5
haz_month = Hazard.from_hdf5(dir_data+'/'+'seasonal_global_wf_2002-2025.hdf')

In [12]:
haz_new_centroids = wf_util.resample_to_fraction(haz_month, target_centroids, threshold, nr_chunks=200)
haz_new_centroids.write_hdf5(dir_data+'/'+'150arcsec_seasonal_global_wf_2002-2025.hdf') 

0 of 200
2025-05-12 13:36:29,148 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 5456226 coordinates.
2025-05-12 13:36:29,513 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 5237 coordinates.
1 of 200
2025-05-12 13:37:11,395 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 5477415 coordinates.
2025-05-12 13:37:11,561 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 175 coordinates.
2 of 200
2025-05-12 13:37:34,100 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 5512067 coordinates.
3 of 200
2025-05-12 13:38:31,802 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 5557726 coordinates.
2025-05-12 13:38:32,399 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 5km for 20585 coordinates.
4 of 200
2025-